In [1]:
from tensorflow.keras import backend as K
import tensorflow_hub as hub
import tensorflow as tf
import pickle
import os
# 设置可使用的 gpu 序号
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
gpus = tf.config.experimental.list_physical_devices('GPU')
# 设置GPU的最大使用率为8192MB
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3000)])
print(gpus)
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TF version:  2.0.0
Hub version:  0.8.0


2023-05-06 08:16:21.056261: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-05-06 08:16:21.062353: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:03:00.0
2023-05-06 08:16:21.062740: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2023-05-06 08:16:21.064190: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2023-05-06 08:16:21.065565: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2023-05-06 08:16:21.066434: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2023-05-06 08:16:21.068086: I tensorflow/stream_executor/plat

In [2]:
from tensorflow.keras.initializers import Orthogonal
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Concatenate, Lambda
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Conv2D, Reshape, Average, Flatten
from tensorflow.keras import metrics
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import h5py
import csv
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#from sklearn.model_selection import train_test_split

## 加载数据集，训练集和测试集，按照数据集大小划定批数据，每一批数据默认为32条数据

In [3]:
import pickle

f = open('../Data/BERT-ServiceDatasetWithNameMiniBatch-TrainData.pickle', 'rb')
traindata = pickle.load(f)
f.close()

In [4]:
import pickle

f = open('../Data/BERT-ServiceDatasetWithNameMiniBatch-TestData.pickle', 'rb')
testdata = pickle.load(f)
f.close()

In [5]:
len(testdata)

70

In [6]:
len(traindata)

273

# 数据集产生器，一条一条的产生，防止一次性全部读入内存，这里只产生训练数据。

In [7]:
maxName = 10
maxDes = None
bert_path = "./tensorflowBert/"

In [8]:
def NameAndDesBertEmbedding():
    # INPUT - Service Name
    input_name_id = Input(shape=(maxName, ),
                          dtype=tf.int32,
                          name="input_word_name_ids")
    input_name_mask = Input(shape=(maxName, ),
                            dtype=tf.int32,
                            name="input_name_masks")
    input_name_segment = Input(shape=(maxName, ),
                               dtype=tf.int32,
                               name="segment_name_ids")
    bert_name_inputs = [input_name_id, input_name_mask, input_name_segment]
    bert_name_layer = hub.KerasLayer(bert_path,
                                     trainable=False,
                                     name="bert_name")
    namePooled_output, nameSequence_output = bert_name_layer(bert_name_inputs)

    input_word_ids = Input(shape=(maxDes, ),
                           dtype=tf.int32,
                           name="input_word_ids")
    input_word_mask = Input(shape=(maxDes, ),
                            dtype=tf.int32,
                            name="input_word_mask")
    input_word_segment = Input(shape=(maxDes, ),
                               dtype=tf.int32,
                               name="input_word_segment")
    bert_description_inputs = [
        input_word_ids, input_word_mask, input_word_segment
    ]

    bert_des_layer = hub.KerasLayer(bert_path,
                                    trainable=False,
                                    name="bert_des")
    # 获取服务名称和服务描述的所有嵌入
    namePooled_output, nameSequence_output = bert_name_layer(bert_name_inputs)
    desPooled_output, desSequence_output = bert_des_layer(
        bert_description_inputs)

    alloutput = [
        namePooled_output, nameSequence_output, desPooled_output,
        desSequence_output
    ]
    modelbert = Model(inputs=[bert_description_inputs, bert_name_inputs],
                      outputs=alloutput)
    return modelbert

In [9]:
modelbert = NameAndDesBertEmbedding()

2023-05-06 08:16:31.210294: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2023-05-06 08:16:31.217209: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2599920000 Hz
2023-05-06 08:16:31.218201: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2af9f7fa0760 executing computations on platform Host. Devices:
2023-05-06 08:16:31.218217: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
2023-05-06 08:16:31.218950: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:03:00.0
2023-05-06 08:16:31.218997: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2023-05-06 08:16:31.219013: I tensorflow/stream_

- 数据组织方式，首先从数据产生器中获取服务描述和服务名称的BERT输入格式，把输入格式数据传入BERT模型获取服务名称、服务描述的词嵌入向量。
- 获取的词嵌入向量既包括名称和描述的整体嵌入，也就是BERT的CLS位置，也包括每个单词的嵌入。
- 获取名称和描述的嵌入以后与标签一起组成元组存入数据文件之中。

In [10]:
def dataAnalysis(all_embs):
    print("这里介绍数据的批次数目")
    print("all_embs batch num is: %d" % len(all_embs))
    print("每条数据中存放的是一个元组，元组长度为4")
    print("type(all_embs[0]) is: %s" % type(all_embs[0]))
    print("tuple all_embs[0]'s length is: %d" % len(all_embs[0]))
    print("第0批数据元组的位置0存放的是原始数据DataFrame")
    print("type(all_embs[0][0]) is: %s" % type(all_embs[0][0]))
    print("the rawData DataFrame data is :")
    print(all_embs[0][0])
    print("第0批数据元组的位置1存放的是服务名称和服务描述的全局嵌入和单词嵌入列表，列表中包含四种嵌入向量，分别是服务名称poolEmbedding 服务名称的sequence_output，服务描述的poolEmbedding，服务描述的sequence_output")
    print("type(all_embs[0][1]) is: %s" % type(all_embs[0][1]))
    print("list of BERTEmbedding all_embs[0][1] length is: %d" % len(all_embs[0][1]))
    print("all_embs[0][1][0] is 第0批数据元组位置1服务嵌入位置0的服务名称的 poolEmbedding，形状为 : " +
          str(all_embs[0][1][0].shape))
    print("all_embs[0][1][1] is 第0批数据元组位置1服务嵌入位置1的服务名称的 sequenceEmbedding ，形状为: " +
          str(all_embs[0][1][1].shape))
    print("all_embs[0][1][2] is 第0批数据元组位置1服务嵌入位置2的服务描述的 poolEmbedding ，形状为: " +
          str(all_embs[0][1][2].shape))
    print("all_embs[0][1][3] is 第0批数据元组位置1服务嵌入位置3的服务描述的 sequenceEmbedding ，形状为: " +
          str(all_embs[0][1][3].shape))
    print("all_embs[0][2] 是第0批数据的BERT嵌入后乘以TF-IWF值取平均后得到的列表，列表中存放%s条numpy数据，每批数据的shape为%s" % (len(all_embs[0][2]) , str(all_embs[0][2][0].shape)))
    print("右侧是该批数据的标签的onehot表示")
    print("all_embs[0][3] 是批数据的标签的onehot表示，形状为 : " + str(all_embs[0][3].shape))

In [11]:
def generator(data):
    while True:
        for d in data:
            data_raw = d[0]
            bertinput_feature = d[1]
            onehot_label = d[2]
            yield data_raw, bertinput_feature,onehot_label

In [15]:
# 这个函数用于把服务描述分词后的token index 换成该index对应的TF-IWF值，并转换为numpy array返回，便于计算嵌入向量权重
def weight2NumpyArray(x1,x2):
    list1 = []
    for token in x1:
        list1.append(x2.get(token))
    length = len(list1)
    arr = np.array(list1)
    # reshape，从数组转换为列向量
    arr = arr.reshape(length,1)
    return arr

In [115]:
def getWeightEmbedding(data):
    data_generator = generator(data)
    epoch = len(data)
    all_embs = []
    for i in range(epoch):
        raw_data, word_data, label_data = next(data_generator)
        # 对DataFrame的'ServiceDes_token_index'列和ServiceDes_token_index_TFIWF列取出得到权重向量
        ServiceDes_token_Weight_Array = raw_data.apply(
            lambda x: weight2NumpyArray(x['ServiceDes_token_index'], x['ServiceDes_token_index_TFIWF']), axis=1)
        epoch_embedding = modelbert.predict(word_data, verbose=1)
        batchWeight_embs = []
        # 使用enumerate遍历一个批次的token_index_Weight
        for i, g in enumerate(ServiceDes_token_Weight_Array):
            # epoch_embedding[3]处是BERT返回的服务描述词Sequence_output
            # 1:len(g)+1,len(g)是服务描述的wordpiece长度，从1开始是避免CLS占位符
            embeddingWeight = epoch_embedding[3][i, 1:len(g)+1, :]*g
#             print(embeddingWeight.shape)
            # 每个词的g权重和每个单词的BERT嵌入相乘，然后得到一句话的整体的平均值
            meanEmbedding = np.mean(embeddingWeight,axis=0)
#             print(meanEmbedding.shape)
            # 把一句话的平均值添加到列表中，一个列表存储32句话，每句话是词向量的平均值
            batchWeight_embs.append(meanEmbedding)
        all_embs.append((raw_data, epoch_embedding,batchWeight_embs,label_data))
    return all_embs

In [116]:
testDataEmbedding = getWeightEmbedding(testdata)

2/2 [==============================] - 0s 10ms/sample


In [118]:
trainDataEmbedding = getWeightEmbedding(traindata)

32/32 [==============================] - 0s 7ms/sample


29/29 [==============================] - 0s 5ms/sample


In [119]:
dataAnalysis(testDataEmbedding)

这里介绍数据的批次数目
all_embs batch num is: 70
每条数据中存放的是一个元组，元组长度为4
type(all_embs[0]) is: <class 'tuple'>
tuple all_embs[0]'s length is: 4
第0批数据元组的位置0存放的是原始数据DataFrame
type(all_embs[0][0]) is: <class 'pandas.core.frame.DataFrame'>
the rawData DataFrame data is :
                                         ServiceName  \
1357                        Open Patent Services API   
14812                                   Pixenate API   
147                    Yahoo Related Suggestions API   
444                                    PhishTank API   
192                                   Cafe Press API   
243              Microsoft Silverlight Streaming API   
14562                                    Xpenser API   
4765                     Snickers Media Encoding API   
1201                                    Campfire API   
14708                                      NN.NF API   
14832                          Vodafone Betavine API   
240                                        MyVox API   
96                

In [120]:
dataAnalysis(trainDataEmbedding)

这里介绍数据的批次数目
all_embs batch num is: 273
每条数据中存放的是一个元组，元组长度为4
type(all_embs[0]) is: <class 'tuple'>
tuple all_embs[0]'s length is: 4
第0批数据元组的位置0存放的是原始数据DataFrame
type(all_embs[0][0]) is: <class 'pandas.core.frame.DataFrame'>
the rawData DataFrame data is :
                                ServiceName  \
14766                              Mapy API   
13581                            Zatune API   
1114                     Webcams.travel API   
660                       bLaugh Comics API   
432                              Yandex API   
632                 Web API for Biology API   
1364                            Whereis API   
15319                        WebResizer API   
427                           SimpleAPI API   
15317                           OpenEco API   
653                             Rakuten API   
824                               FavQs API   
1388                           Prodigem API   
638                           AgentRank API   
9283   Twinword Category Recommendation 

## 接下来保存文本嵌入数据，保存后再读取出来看看是否正确

In [121]:
import pickle
fw1 = open('../Data/BertEmbeddingWithNameAndDescriptionbyMiniBatch-TrainData.pickle', 'wb')
fw2 = open('../Data/BertEmbeddingWithNameAndDescriptionbyMiniBatch-TestData.pickle', 'wb')
pickle.dump(trainDataEmbedding, fw1)
pickle.dump(testDataEmbedding, fw2)
fw1.close()
fw2.close()

In [122]:
f1 = open('../Data/BertEmbeddingWithNameAndDescriptionbyMiniBatch-TrainData.pickle', 'rb')
f2 = open('../Data/BertEmbeddingWithNameAndDescriptionbyMiniBatch-TestData.pickle', 'rb')
traindataLoaded = pickle.load(f1)
testdataLoaded = pickle.load(f2)
f1.close()
f2.close()

In [123]:
dataAnalysis(traindataLoaded)

这里介绍数据的批次数目
all_embs batch num is: 273
每条数据中存放的是一个元组，元组长度为4
type(all_embs[0]) is: <class 'tuple'>
tuple all_embs[0]'s length is: 4
第0批数据元组的位置0存放的是原始数据DataFrame
type(all_embs[0][0]) is: <class 'pandas.core.frame.DataFrame'>
the rawData DataFrame data is :
                                ServiceName  \
14766                              Mapy API   
13581                            Zatune API   
1114                     Webcams.travel API   
660                       bLaugh Comics API   
432                              Yandex API   
632                 Web API for Biology API   
1364                            Whereis API   
15319                        WebResizer API   
427                           SimpleAPI API   
15317                           OpenEco API   
653                             Rakuten API   
824                               FavQs API   
1388                           Prodigem API   
638                           AgentRank API   
9283   Twinword Category Recommendation 

In [124]:
dataAnalysis(testdataLoaded)

这里介绍数据的批次数目
all_embs batch num is: 70
每条数据中存放的是一个元组，元组长度为4
type(all_embs[0]) is: <class 'tuple'>
tuple all_embs[0]'s length is: 4
第0批数据元组的位置0存放的是原始数据DataFrame
type(all_embs[0][0]) is: <class 'pandas.core.frame.DataFrame'>
the rawData DataFrame data is :
                                         ServiceName  \
1357                        Open Patent Services API   
14812                                   Pixenate API   
147                    Yahoo Related Suggestions API   
444                                    PhishTank API   
192                                   Cafe Press API   
243              Microsoft Silverlight Streaming API   
14562                                    Xpenser API   
4765                     Snickers Media Encoding API   
1201                                    Campfire API   
14708                                      NN.NF API   
14832                          Vodafone Betavine API   
240                                        MyVox API   
96                

- all_embs[0][1][0] is 服务名称的 poolEmbedding，形状为 : (32, 768)
- all_embs[0][1][1] is 服务名称的 sequenceEmbedding ，形状为: (32, 10, 768)
- all_embs[0][1][2] is 服务描述的 poolEmbedding ，形状为: (32, 768)
- all_embs[0][1][3] is 服务描述的 sequenceEmbedding ，形状为: (32, 110, 768)